### IMPORTS

In [1]:
import ipynb.fs.full.processing as processing
import ipynb.fs.full.features as features
import ipynb.fs.full.training as training
import ipynb.fs.full.analysis as analysis
import ipynb.fs.full.storage as storage

### SERIALIZE THE YAML CONFIG

In [2]:
config = storage.load_config('extra/config.yaml')

In [3]:
config

{'data': 'extra/dataset.csv',
 'processing': {'resample': True,
  'time': 'D',
  'aggregate': {'Open': 'first',
   'High': 'max',
   'Low': 'min',
   'Close': 'last',
   'Volume': 'sum'},
  'label': {'from': 'Close', 'shift': 1}},
 'features': {'add': ['SK', 'SD', 'Momentum'],
  'window': 14,
  'filter': ['Close', 'Momentum', 'SD', 'Label']},
 'ensamble': {'models': [{'linreg': {'split': {'train': 0.8, 'test': 0.2}}},
   {'lstm': {'split': {'train': 0.6, 'test': 0.2, 'validation': 0.2},
     'layers': [{'lstm': {'value': 120, 'activation': 'relu'}},
      {'dropout': {'value': 0.15}},
      {'dense': {'value': 50, 'activation': 'relu'}},
      {'dense': {'value': 1}}],
     'epochs': 20,
     'batch': 8,
     'validation': 25,
     'loss': 'mean_squared_error',
     'optimizer': 'rmsprop'}}],
  'decision': 'mean'},
 'training': {'linreg': None,
  'lstm': {'layers': [{'lstm': {'nodes': 120, 'activation': 'relu'}},
    {'dropout': {'nodes': 0.15}},
    {'dense': {'nodes': 50, 'activation

### STEP 1: PROCESS RAW DATA TO DATAFRAME

In [4]:
dataframe = processing.create_dataframe(config)

In [5]:
dataframe.head(5)

,Open,High,Low,Close,Volume,Label
Date_Timestamp,,,,,,
2010-01-01,1.43327,1.43356,1.43207,1.43335,39761.000053,1.43141
2010-01-03,1.43024,1.43359,1.42951,1.43141,3001.600003,1.44244
2010-01-04,1.43143,1.44556,1.42559,1.44244,80019.400094,1.43634
2010-01-05,1.44238,1.44834,1.43445,1.43634,79887.100067,1.44005
2010-01-06,1.43638,1.44342,1.42807,1.44005,80971.800085,1.43155


### STEP 2: ADD FEATURES TO DATAFRAME

In [6]:
dataset = features.add(dataframe, config['features'])

In [7]:
dataset.head(5)

,Close,Momentum,SD,Label
Date_Timestamp,,,,
2010-02-01,1.39200,-0.04649,14.842308,1.39737
2010-02-02,1.39737,-0.03928,13.972612,1.38969
2010-02-03,1.38969,-0.05070,11.302307,1.37395
2010-02-04,1.37395,-0.05393,11.014429,1.36763
2010-02-05,1.36763,-0.04356,11.636518,1.36409


### STEP 3: TRAIN ENSAMBLE

In [8]:
ensamble = []

In [9]:
# LOOP THROUGH REQUESTED MOEDLS
for item in config['ensamble']['models']:
    
    # MODEL PROPS
    name = list(item)[0]
    params = item[name]
    
    # GENERATE FORMATTED, SPLIT & NORMALIZED DATA FOR TRAINING
    data = features.split(dataset, name, params)

    # TRAIN THE MODEL
    model = training.start(data, name, params)
    
    # APPEND IT TO THE ENSAMBLE
    ensamble.append(model)

LINREG STARTED
LINREG FINISHED
LSTM STARTED
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 1861 samples, validate on 620 samples
Epoch 1/20
1861/1861 [==============================] - 1s 674us/sample - loss: 0.0245 - val_loss: 0.0040
Epoch 2/20
1861/1861 [==============================] - 1s 370us/sample - loss: 0.0056 - val_loss: 0.0044
Epoch 3/20
1861/1861 [==============================] - 1s 371us/sample - loss: 0.0043 - val_loss: 0.0014
Epoch 4/20
1861/1861 [==============================] - 1s 366us/sample - loss: 0.0034 - val_loss: 0.0024
Epoch 5/20
1861/1861 [==============================] - 1s 369us/sample - loss: 0.0029 - val_loss: 0.0011
Epoch 6/20
1861/1861 [==============================] - 1s 380us/sample - loss: 0.0026 - val_loss: 0.0011
Epoch 7/20
1861/1861 [==============================] - 1s 379us/sample - loss: 0.0023 - val_loss:

### STEP 5: SAVE THE ENSAMBLE

In [ ]:
storage.save_result(result, config)